### Importing the Features and target

In [1]:
# The code was removed by Watson Studio for sharing.

,0,1
0,3.316751,-1.443463
1,2.209465,0.333393
2,2.516740,-1.031151
3,3.757066,-2.756372
4,1.008908,-0.869831


In [2]:

body = client_951162b69b3d4dfa95adbc4e5b9a3121.get_object(Bucket='hdppcamodel-donotdelete-pr-jziye9gx40rkg8',Key='target.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
    
target = pd.read_csv(body)
target.head()
#import os, pandas as pd
#target = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/target.csv')

,0
0,1
1,1
2,1
3,1
4,1


In [3]:
X = features.iloc[:,1:].values
X[0:2]

array([[-1.44346263],
       [ 0.33339289]])

In [4]:
y = target['0'].values
y[0:5]

array([1, 1, 1, 1, 1])

### Splitting the X and y into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Fitting Logistic Regression to the Training set

In [6]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
mymodel=classifier.fit(X_train, y_train)

### Predicting the Test set results

In [7]:
y_pred = classifier.predict(X_test)
y_pred[0:5]

array([1, 3, 2, 3, 2])

### Making the Confusion Matrix

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 7,  0,  7],
       [ 1, 15,  0],
       [ 3,  0,  3]])

### Saving the Model

In [9]:
#from dsx_ml.ml import save

#save(name='PCAModel',
#     model=classifier,
#     x_test=pd.DataFrame(X_test),
#     y_test=pd.DataFrame(y_test),
#     algorithm_type='Classification')

In [10]:
apikey = 'PKK-we6c5yXAirsmVgYS0ntKzP8lcrckK5wp1ECHnKF4'
location = 'eu-gb'

wml_credentials = {
    "apikey": apikey,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [11]:
#!pip install -U ibm-watson-machine-learning

In [12]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [13]:
COS_API_KEY_ID = "8EBPCWptKaOhWXlWnS_I-DQODf9bUHcMGS9Hxxsr6gFC"
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/61e4e1eeb97c8ff1d896b06f1798fc80:951162b6-9b3d-4dfa-95ad-bc4e5b9a3121::" 
COS_ENDPOINT = "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints"
IAM_AUTH_ENDPOINT = "s3.ap.cloud-object-storage.appdomain.cloud"
BUCKET_NAME = "mynewbucket"

In [14]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=IAM_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [15]:
training_data_file_name = "Wine.csv"

In [16]:
MODEL_NAME = "Wine Model"
DEPLOYMENT_NAME = "Wine Model Deployment"

In [17]:
software_spec_uid = client.software_specifications.get_id_by_name("default_py3.8")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
    client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

Software Specification ID: ab9e1b80-f2ce-592c-a7d2-4f2344f77194


In [20]:
print("Storing model ...")

published_model_details = client.repository.store_model(model=mymodel, meta_props=model_props, training_data=X_train, training_target=y_train)

model_uid = client.repository.get_model_uid(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...


WMLClientError: Publishing model failed.
Reason: You must set the space/project id. Use `client.set.default_space(<SPACE_UID>)`/`client.set.default_project(<PROJECT_UID>)` to proceed.